In [1]:
import pandas as pd
import numpy as np
import os
import re

import nltk
from numpy.random import randint, seed
from sklearn.feature_extraction.text import CountVectorizer

import konlpy
from konlpy.tag import Twitter

from collections import Counter
from wordcloud import WordCloud

from PIL import Image  
from IPython.display import set_matplotlib_formats
from nltk.corpus import stopwords

## 1. 교양오락문화생활비

### Load data

In [ ]:
os.chdir(r"C:\Users\Gram\Desktop\네이버뉴스 립스틱효과\3. 뉴스데이터\7. 네이버 뉴스 헤드라인, 본문요약 크롤링 통합본")

In [4]:
my_text=pd.read_excel("교양오락문화생활비.xlsx", encoding='CP949')
my_text.shape

(13738, 3)

In [5]:
my_text['title'].values[:10]

array(['김준호 유튜브 채널 구독자 38만명 돌파…뼈그맨 인증',
       '\'두데\' 감스트 "구독자들 \'야\'라고 불러…난 \'인직아\'라고 불린다"',
       '[뉴스레터 구독 이벤트] 1차 당첨자 추첨 과정 공개…100명의 행운의 주인공은...',
       '[MT리포트] "1년 수입 10억"…수백만 구독자 \'유튜버의 세계\'',
       'CJ E&M, ‘다이아 티비’ 구독자 1억 6000만명 돌파',
       "'슈가맨' '히든싱어' 등 음악예능 인기에…JTBC 유튜브 구독자 200만 돌파",
       "250만 구독자 '유튜버 망치'한국의 맛 알려",
       "[뉴스레터 구독 이벤트] RC카 '러커스'로 대리만족 한 번 해보시렵니까",
       "[뉴스레터 구독 이벤트] 1차를 놓쳤다면 'V30'가 기다리는 2차 이벤트에 도전",
       '250만 구독자 유튜버가 전하는 한국의 맛'], dtype=object)

### 전처리

In [6]:
len(my_text['title'].values) - len(set(my_text['title'].values)) # 중복 데이터

772

In [7]:
no_meaning = "구독|책|핫플레이스|핫|플레이스|레이스|휴가비"
my_text_clean = []

for a_line in set(my_text['title'].values):
    a_line = re.sub('[^ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)           # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)               # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [8]:
my_text_clean[:10]

['서울시교육청 은혜초 무단폐교 이사장 고발 교장은 해임 요청',
 '다시 통상임금을 묻다',
 '경남과기대 용역근로자에 임명장 수여',
 ' 과 길 외국어 란 렌즈를 통해 본 인류 ',
 ' 너도 인간이니 공승연 인공지능 사랑 할 수 있어 ',
 ' 김비서가 왜 그럴까 박서준 박민영 심야 놀이공원 데이트 달달 로맨틱 ',
 '커넥츠북 매일 매일 드림 명 추첨 공짜로',
 '최순실 메모 보니 특활비 배분에 개입',
 ' 이동진의 빨간방 인터뷰 방식 때문에 의미도 상당한 ',
 ' 꽂이 새 자 코끼리의 시간 쥐의 시간 ']

In [9]:
hannanum = konlpy.tag.Hannanum()

# 명사 추출
my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)
    
#my_words[:10]

In [10]:
# 단음절 제거
my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word
        
#my_words_2

### 2-gram

In [11]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [12]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [13]:
res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('정규직 전환', 103),
 ('국정원 특활비', 73),
 ('근로자 휴가지원사업', 60),
 ('만명 돌파', 56),
 ('임금협약 체결', 50),
 ('박서준 박민영', 42),
 ('김비서 박서준', 35),
 ('박근혜 국정원', 34),
 ('노사 임단협', 33),
 ('만원 지원', 33)]

### 다음에 나올 연관 단어 확인

In [14]:
def predict(text):
    n = 0
    print(text, "--->", end=" ")
    for i in range(len(res_dict)):
        if text == res_dict[i][0].split(" ")[0]:
            print(res_dict[i][0].split(" ")[1], end=" ")
            n += 1
        if n == 5:
            break
    print("")

In [15]:
def predict_arr(arr):
    n = range(len(arr))
    for i, j in zip(arr, n):
        print(j+1, "위 : ", end="")
        predict(i)

In [16]:
predict_arr(['여행', '맛집', '종합', '오늘', '개최', 
           '서울', '부산', '올해', '제주', '공개'])

1 위 : 여행 ---> 코스 어디 이야기 이태 향기 
2 위 : 맛집 ---> 기행 대천왕 샤이바 경기 공개 
3 위 : 종합 ---> 밤도깨비 포토 검찰 김비서 숲속 
4 위 : 오늘 ---> 첫방 박근혜 세계 방송 공개 
5 위 : 개최 ---> 금감원 대통령 밤도깨비 여름 연휴 
6 위 : 서울 ---> 성수동 강남대로 상권 강남 마지막 
7 위 : 부산 ---> 보이스피싱 관광 기장 송도구름산 국제금융센터 
8 위 : 올해 ---> 선정 선포식 특집 인기 협력사 
9 위 : 제주 ---> 호텔 동네방 땅값 중국인 가동 
10 위 : 공개 ---> 밤도깨비 삼성 스포티한 이재명 가을하늘 


## 2. 교통통신비

In [17]:
my_text = pd.read_excel("교통통신비.xlsx", encoding='CP949')

no_meaning = "교통비|대중교통|통신비"
my_text_clean = []

for a_line in set(my_text['title'].values):
    a_line = re.sub('[^ ㄱ-ㅣ가-힣0-9]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)            # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)                # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [18]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)

my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [19]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('최저임금 산입범위', 138),
 ('서울 무료', 122),
 ('출퇴근 무료', 101),
 ('가계 정책협의회', 96),
 ('미세먼지 무료', 93),
 ('서울시 미세먼지', 90),
 ('미세먼지 비상저감조치', 88),
 ('보편요금제 도입', 84),
 ('무료 미세먼지', 70),
 ('서울시 무료', 70)]

In [20]:
def predict(text):
    n = 0
    print(text, "--->", end=" ")
    for i in range(len(res_dict)):
        if text == res_dict[i][0].split(" ")[0]:
            print(res_dict[i][0].split(" ")[1], end=" ")
            n += 1
        if n == 5:
            break
    print("")

def predict_arr(arr):
    n = range(len(arr))
    for i, j in zip(arr, n):
        print(j+1, "위 : ", end="")
        predict(i)

In [21]:
predict_arr(['지원', '보편요금제', '정부', '이통사', '추진',
          '이통3사', '이용', '종합', '최저임금', '서울'])

1 위 : 지원 ---> 확대 공약 미세먼지 최저임금 보편요금제 
2 위 : 보편요금제 ---> 도입 규개위 국회 이통사 심사 
3 위 : 정부 ---> 이통사 보편요금제 1년 청년 압박 
4 위 : 이통사 ---> 보편요금제 요금제 정부 고가요금제 알뜰폰 
5 위 : 추진 ---> 보편요금제 서울시 종합 최저임금 경기 
6 위 : 이통3사 ---> 보편요금제 갤럭시 실적 씽큐 요금제 
7 위 : 이용 ---> 미세먼지 캠페인 당부 싼커 가능 
8 위 : 종합 ---> 최저임금 미세먼지 가계 보편요금제 텔레콤 
9 위 : 최저임금 ---> 산입범위 인상 산입 상여금 개정안 
10 위 : 서울 ---> 무료 출퇴근 미세먼지 버스 하루 


## 3. 내구재비

In [22]:
my_text = pd.read_excel("내구재비.xlsx", encoding='CP949')

no_meaning = "가전제품|가전|내구재|전자제품|전자|제품"
my_text_clean = []

for a_line in set(my_text['title'].values):
    a_line = re.sub('[^ ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)            # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)                # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [23]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)

my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [24]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('생산 소비', 113),
 ('시장 공략', 97),
 ('소비 트리플', 56),
 ('출시 삼성', 50),
 ('최고 대형', 47),
 ('삼성 건조기', 42),
 ('올해 최고', 38),
 ('삼성 빅스비', 35),
 ('개월 연속', 34),
 ('국내 출시', 33)]

In [25]:
def predict(text):
    n = 0
    print(text, "--->", end=" ")
    for i in range(len(res_dict)):
        if text == res_dict[i][0].split(" ")[0]:
            print(res_dict[i][0].split(" ")[1], end=" ")
            n += 1
        if n == 5:
            break
    print("")
    
def predict_arr(arr):
    n = range(len(arr))
    for i, j in zip(arr, n):
        print(j+1, "위 : ", end="")
        predict(i)

In [26]:
predict_arr(['삼성', '출시', '종합', '스마트폰', '미세먼지',
          '생산', '공개', '수출', '시장', '올해'])

1 위 : 삼성 ---> 건조기 빅스비 갤럭시 사장 샘표 
2 위 : 출시 ---> 삼성 대우 스마트홈 만원 수출 
3 위 : 종합 ---> 삼성 생산 미세먼지 기재부 김현석 
4 위 : 스마트폰 ---> 시장 씽큐 보상 분기 출시 
5 위 : 미세먼지 ---> 관련 영향 황사 분기 공기청정기 
6 위 : 생산 ---> 소비 증가세 반도체 개월 조정 
7 위 : 공개 ---> 삼성 중국 대유위니 생산 정부 
8 위 : 수출 ---> 호조 제조업 중심 감소 생산 
9 위 : 시장 ---> 공략 진출 삼성 연속 정조준 
10 위 : 올해 ---> 최고 분기 소비 경제성장률 성장률 


## 4. 외식비

In [27]:
my_text = pd.read_excel("외식비.xlsx", encoding='CP949')

no_meaning = "외식|외식비|업계"
my_text_clean = []

for a_line in set(my_text['title'].values):
    a_line = re.sub('[^ ㄱ-ㅣ가-힣0-9]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)            # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)                # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [28]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)

my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [29]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('최저임금 인상', 99),
 ('가격 인상', 57),
 ('4월 소비자물가', 28),
 ('김밥 자장면', 26),
 ('편의점 도시락', 25),
 ('김영록 장관', 23),
 ('소비자물가 상승', 21),
 ('송재희 지소연', 21),
 ('식품 기업', 20),
 ('창업 인큐베이팅', 20)]

In [30]:
def predict(text):
    n = 0
    print(text, "--->", end=" ")
    for i in range(len(res_dict)):
        if text == res_dict[i][0].split(" ")[0]:
            print(res_dict[i][0].split(" ")[1], end=" ")
            n += 1
        if n == 5:
            break
    print("")
    
def predict_arr(arr):
    n = range(len(arr))
    for i, j in zip(arr, n):
        print(j+1, "위 : ", end="")
        predict(i)

In [31]:
predict_arr(['상승', '프랜차이즈', '최저임금', '메뉴', '식품', 
             '업체', '창업', '최저임금', '정부', '브랜드'])

1 위 : 상승 ---> 최저임금 정부 즉석식품 편의점 6개월 
2 위 : 프랜차이즈 ---> 창업 업체 박람회 창업박람회 21 
3 위 : 최저임금 ---> 인상 여파 영향 편승 부메랑 
4 위 : 메뉴 ---> 출시 13종 개발 1위 가격 
5 위 : 식품 ---> 기업 산업 생필품 분야 가격 
6 위 : 업체 ---> 대표 10곳 사용 77 가격 
7 위 : 창업 ---> 인큐베이팅 아이템 시장 지원 브랜드 
8 위 : 최저임금 ---> 인상 여파 영향 편승 부메랑 
9 위 : 정부 ---> 최저임금 1년 가격 오징어 체감물가 
10 위 : 브랜드 ---> 경복궁 가정 대상 리포터 미스터보쌈 


## 5. 의료보건비

In [32]:
my_text = pd.read_excel("의료보건비.xlsx", encoding='CP949')

no_meaning = "건강보험|건강|보험|의료|의료비|병원|병원비"
my_text_clean = []

for a_line in set(my_text['title'].values):
    a_line = re.sub('[^ ㄱ-ㅣ가-힣0-9]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)            # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)                # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [33]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)

my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [34]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('문재 케어', 204),
 ('보장성 강화', 139),
 ('고위험 임산부', 55),
 ('9988 출시', 45),
 ('국민 일산', 45),
 ('농협생명 9988', 43),
 ('임산부 지원', 38),
 ('종합 3인실', 38),
 ('상복부 초음파', 36),
 ('가입 가능', 34)]

In [35]:
def predict(text):
    n = 0
    print(text, "--->", end=" ")
    for i in range(len(res_dict)):
        if text == res_dict[i][0].split(" ")[0]:
            print(res_dict[i][0].split(" ")[1], end=" ")
            n += 1
        if n == 5:
            break
    print("")
    
def predict_arr(arr):
    n = range(len(arr))
    for i, j in zip(arr, n):
        print(j+1, "위 : ", end="")
        predict(i)

In [36]:
predict_arr(['지원', '문재', '정부', '의협', '환자', 
             '복지부', '보장성', '국민공단', '국민', '공단'])

1 위 : 지원 ---> 확대 협약 사업 강화 내년 
2 위 : 문재 ---> 케어 대통령 정부 헬스케어 강아지분양 
3 위 : 정부 ---> 1년 출범 지원 대납 보장성 
4 위 : 의협 ---> 건정심 비대위 복지부 문재 문케 
5 위 : 환자 ---> 급증 지원 사랑 실손 행복 
6 위 : 복지부 ---> 의협 장관 권고 보장성 상복부 
7 위 : 보장성 ---> 강화 강화대책 확대 강화정책 대책 
8 위 : 국민공단 ---> 이사장 이달 채용 일산장 부산중부지사 
9 위 : 국민 ---> 일산 부담 10명 의견 종합계획 
10 위 : 공단 ---> 이사장 일산 상임감사 올해 정책연구원장 


## 6. 의류비

In [37]:
my_text = pd.read_excel("의류비.xlsx", encoding='CP949')

no_meaning = "의류|패션"
my_text_clean = []

for a_line in set(my_text['title'].values):
    a_line = re.sub('[^ ㄱ-ㅣ가-힣0-9]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)            # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)                # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [38]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)

my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [39]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('공항 포토', 108),
 ('시각 코스닥', 37),
 ('코스닥 코스닥', 37),
 ('코스닥 현재', 37),
 ('엑소 세훈', 32),
 ('미모 공항', 29),
 ('비주얼 공항', 27),
 ('화보 공개', 27),
 ('최대 50', 25),
 ('트롬 미니워시', 24)]

In [40]:
def predict(text):
    n = 0
    print(text, "--->", end=" ")
    for i in range(len(res_dict)):
        if text == res_dict[i][0].split(" ")[0]:
            print(res_dict[i][0].split(" ")[1], end=" ")
            n += 1
        if n == 5:
            break
    print("")
    
def predict_arr(arr):
    n = range(len(arr))
    for i, j in zip(arr, n):
        print(j+1, "위 : ", end="")
        predict(i)

In [41]:
predict_arr(['공항', '포토', '브랜드', '화보', '여름', 
             '업계', '개최', '출시', '진행', '백화점'])

1 위 : 공항 ---> 포토 트와이스 코웨 방탄소년단 업계 
2 위 : 포토 ---> 엑소 현아 안소희 구혜선 세븐틴 
3 위 : 브랜드 ---> 모카썸 론칭 모델 쿠스토 데이즈 
4 위 : 화보 ---> 공개 공항 심소영 포토 고준희 
5 위 : 여름 ---> 상품 화보 기획전 바캉스 뷰티 
6 위 : 업계 ---> 최초 진출 1분기 냉감 사업 
7 위 : 개최 ---> 포토 수익금 롯데 오마이걸 삼성 
8 위 : 출시 ---> 포토 봇물 백화점 용량 글로벌 
9 위 : 진행 ---> 포토 2018 업계 화보 공승연 
10 위 : 백화점 ---> 겨울 마트 매장 명품 아웃도어 
